In [1]:
import pandas as pd

def main():
    
    files =["transactions_log_201702","transactions_log_201701","transactions_log_201612","transactions_log_201611","transactions_log_201610","transactions_log_201609"]
    
    #combining transactions log
    df0 = pd.concat([pd.read_csv('./data/' + f + '.csv') for f in files])
    df0 = df0.sort_values(['msno','transaction_date','membership_expire_date'])
    
    #Aggregating transactions log for feature creation
    #aggregating columns for each msno
    df1 = df0.groupby('msno').agg({'payment_plan_days':['mean'], 'actual_amount_paid':['mean'], 
                                    'plan_list_price':['mean'],'payment_method_id':'count'}).reset_index()

    df1.columns = ['msno', 'payment_plan_days_mean','actual_amount_paid_mean','plan_list_price_mean','payment_counts']

    df1.columns = ['msno'] + [str(col) for col in df1.columns[1:]] 
    
    
    #Getting the latest record for each msno
    df2 = df0.loc[df0.groupby('msno').transaction_date.idxmax()].reset_index(drop=True)
    
    df2.columns = ['msno','latest_payment_method_id','latest_payment_plan_days',
               'latest_plan_list_price_mean','latest_actual_amount_paid','latest_is_auto_renew',
               'latest_trnsaction_date','latest_membership_expire_date','latest_is_cancel']

    df2.columns = ['msno'] + [str(col) for col in df2.columns[1:]] 
    
    #merging the two dataframes
    df = pd.merge(df1, df2, how='left', left_on=['msno'], right_on=['msno'])
    
    #writing the combined aggregated transaction_logs to csv
    df.to_csv("transaction_logs_initial1.csv",index=False)
    
    
if __name__ == '__main__':
    main()

In [4]:
df = pd.read_csv("transaction_logs_initial1.csv")

In [5]:
df.head()

,msno,payment_plan_days_mean,actual_amount_paid_mean,plan_list_price_mean,payment_counts,latest_payment_method_id,latest_payment_plan_days,latest_plan_list_price_mean,latest_actual_amount_paid,latest_is_auto_renew,latest_trnsaction_date,latest_membership_expire_date,latest_is_cancel
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.0,0.0,0.0,1,35,7,0,0,0,20160909,20160914,0
1,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.0,0.0,0.0,1,35,7,0,0,0,20160909,20160914,0
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,7.0,0.0,0.0,1,35,7,0,0,0,20160909,20160914,0
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,395.0,1599.0,1599.0,1,22,395,1599,1599,0,20161023,20180206,0
4,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30.0,99.0,99.0,4,41,30,99,99,1,20170215,20170315,0
